In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
from pprint import pprint
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import pandas as pd
import urllib.request

In [2]:
NUM_THREAD = 20
EXECUTOR = ThreadPoolExecutor(max_workers=NUM_THREAD)

In [3]:
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_prefs = {
      'profile.managed_default_content_settings.images': 2,
    }
chrome_options.add_experimental_option('prefs', chrome_prefs)

DRIVERS = []
for i in range(NUM_THREAD):
    DRIVERS.append(webdriver.Chrome(options=chrome_options))

In [4]:
LOGFILE = open('log.txt', 'a')
def log(msg):
    LOGFILE.write(msg + '\n')
    LOGFILE.flush()

In [5]:
FILENAME = 'game_data.csv'
df = pd.read_csv(FILENAME)

In [6]:
df

,id,url,app_name,star_percent,good_percent,install_count,update_time,size,comment_count,num_screenshot,tags,editor_comment,description_text,inner_page_crawled
0,com.miHoYo.bh3.wdj,https://www.wandoujia.com/apps/com.miHoYo.bh3.wdj,崩坏3,84%,77.00%,218.1万,2019/03/14,821.31MB,940,5,"网络游戏, 动作射击,",从 Q 版 2D 到 3D 精模，从横版射击到全视角动作格斗，「崩坏 3」打破了前作的很多设...,【战斗伙伴 武装人偶登场】\n来自量子之海的谜之物种，武器所化身的人偶参战！第十章主线「深渊...,True
1,com.tencent.tmgp.dragonnest,https://www.wandoujia.com/apps/com.tencent.tmg...,龙之谷,74%,62.00%,887.7万,2018/03/09,1.1GB,43,5,"动作冒险, 角色扮演,",为了阿尔特里亚，一起去冒险！,《龙之谷》手游是一款由腾讯运营，以端游《龙之谷》为故事背景制作的3DARPG手游。游戏原汁原...,True
2,com.tencent.tmgp.tstl,https://www.wandoujia.com/apps/com.tencent.tmg...,天龙八部手游,78%,54.00%,768.6万,2019/01/16,1.4GB,20,5,"网络游戏, RPG,",十年情怀，经典复刻，力求还原最原汁原味的武侠RPG体验,《天龙八部》手游是一款全新3DMMO武侠手游，由“天龙八部”端游原班人马精心打造、腾讯代理的...,True
3,com.netease.gmdl.uc,https://www.wandoujia.com/apps/com.netease.gmd...,光明大陆,76%,64.00%,366.8万,2018/11/27,1.08GB,28,5,"网络游戏, RPG,",不是电影，胜似电影，可以玩的魔幻3D大片。,影中之刃，暗影随行！《光明大陆》年度资料片“黯月之潮”上线。为对抗恶魔入侵，潜藏于阴影中的古...,True
4,com.snailgame.jysgqy.aligames,https://www.wandoujia.com/apps/com.snailgame.j...,九阳神功：起源,40%,50.00%,71.9万,2019/03/06,673.13MB,3,5,"网络游戏, 动作射击,",英雄战场谁主沉浮，操控为王唯我独尊！,【主机工艺3DMMO手游《九阳神功：起源》，“九阳神功”原班百人研发团队匠心打造，2017年...,True
5,com.netease.ma71.aligames,https://www.wandoujia.com/apps/com.netease.ma7...,边境之旅,68%,51.00%,308.5万,2018/08/08,443.32MB,131,5,"动作冒险, 角色扮演,",徒步冒险主题休闲经营类游戏，画风清新可人，永远在路上的你，诗与远方并不遥远。,海外成绩\n游戏已经在海外市场上线，并在上线当日就获得App Store全球热门推荐。上线后...,True
6,gamelab3d.com.FreeRoam3D,https://www.wandoujia.com/apps/gamelab3d.com.F...,3D自由城：卧底 FreeRoam3D,84%,51.00%,343.6万,2014/09/02,44.68MB,8,5,"动作冒险, 角色扮演,",游戏中可以驾驶各种车辆，包括汽车，卡车，客车四轮摩托车甚至是船和飞机。还可以使用各种武器，猎...,《3D自由城：卧底 Free Roam 3D: Undercover》是一款动作游戏。游戏各...,True
7,com.orange.org_player_new_alone2662.uc,https://www.wandoujia.com/apps/com.orange.org_...,穿越之姻缘劫,74%,61.00%,121.5万,2018/04/28,130.85MB,82,5,"动作冒险, 角色扮演,",只因命格星君那老糊涂写错了一笔,只因命格星君那老糊涂写错了一笔，让本该出生在古代的你出生在了现代，现在那个老糊涂发现了这个错...,True
8,com.xmcy.hykb,https://www.wandoujia.com/apps/com.xmcy.hykb,好游快爆,84%,71.00%,1722.9万,2019/01/30,9.79MB,24,5,"辅助工具, 游戏助手,",游戏爱好者交流社区，可以抢先体验新游，时刻关注排行榜掌握手游市场动向,好游快爆，游戏爱好者自己的APP，所有游戏均来自玩家分享。\n一起品鉴精品好游，抢先体验内测...,True
9,com.xcpsp.zssbdyl.mt,https://www.wandoujia.com/apps/com.xcpsp.zssbd...,战神：斯巴达幽灵,80%,65.00%,31.5万,2018/07/05,1.1GB,17,5,"动作冒险, 角色扮演,",unknown,由Dawn和SCE开发出版的第三人动作冒险游戏。 它于2010年11月2日首次发布于PSP掌...,True


In [12]:
QUEUE = []
for i in range(len(df)):
    if df['inner_page_crawled'][i] == False or df['app_name'][i] == 'unknown':
        QUEUE.append(i)
len(QUEUE)

3

In [8]:
DONE = 0

In [9]:
def crawl_inner_page(index):
    
    
    global DRIVERS
    while len(DRIVERS) == 0:
        time.sleep(0.1)
    driver = DRIVERS.pop(0)
    
    # open page
    appid = df['id'][index]
    url = df['url'][index]
    driver.get(url)
    wait = WebDriverWait(driver, 3)
    driver.execute_script("window.stop();")
    
    # app name
    try:
        title_frame = driver.find_element_by_class_name('app-name').find_element_by_class_name('title')
        appname = title_frame.text
    except:
        appname = 'unknown'
    
    # rating
    try:
        starpercent = driver.find_element_by_class_name('avg-score-current').get_attribute('style')
        starpercent = starpercent.replace('width:','').replace(';','').strip()
    except:
        starpercent = 'unknown'
    
    # install count
    try:
        installcount = driver.find_element_by_class_name('app-info-data')\
                             .find_elements_by_tag_name('span')[0].find_elements_by_tag_name('i')[0].text
    except:
        installcount = 'unknown'
    
    # 好评率
    try:
        goodpercent = driver.find_element_by_class_name('app-info-data')\
                             .find_elements_by_tag_name('span')[1].find_elements_by_tag_name('i')[0].text
    except:
        goodpercent = 'unknown'
    
    # update time
    try:
        updatetime = driver.find_element_by_class_name('update-time').text
        udpatetime = u'' + updatetime  # convert to unicode string before replace
        updatetime = updatetime.replace('时间:','').strip()
    except:
        updatetime = 'unknown'
    
    # size
    try:
        size = driver.find_element_by_class_name('infos-list').find_elements_by_tag_name('dd')[0].text
    except:
        size = 'unknown'
    
    # comment count
    try:
        cmtcount = driver.find_element_by_class_name('app-info-data')\
                         .find_elements_by_tag_name('a')[0].find_elements_by_tag_name('i')[0].text
    except:
        cmtcount = 'unknown'
    
    # tags 
    try:
        tagboxes = driver.find_element_by_class_name('tag-box').find_elements_by_tag_name('a')
        tags = u''
        for tagbox in tagboxes:
            tags += tagbox.text + ', '
    except:
        tags = 'unknown'
    
    # editor comment
    try:
        editorcomment = driver.find_element_by_class_name('editorComment').text
        editorcomment = u'' + editorcomment
        editorcomment = editorcomment.replace('小编点评','').strip()
    except:
        editorcomment = 'unknown'
    
    # number of screenshot available in description page
    try:
        numscreenshot = driver.find_element_by_class_name('view-box')\
                              .find_elements_by_tag_name('div')[0].get_attribute('data-length')
    except:
        numscreenshot = 'unknown'
    
    # description
    try:
        description = driver.find_element_by_class_name('desc-info')\
                            .find_elements_by_tag_name('div')[0].find_elements_by_tag_name('div')[0].text
    except:
        description = 'unknown'
    
    df.at[index, 'app_name'] = appname
    df.at[index, 'star_percent'] = starpercent
    df.at[index, 'good_percent'] = goodpercent
    df.at[index, 'install_count'] = installcount
    df.at[index, 'update_time'] = updatetime
    df.at[index, 'size'] = size
    df.at[index, 'comment_count'] = cmtcount
    df.at[index, 'num_screenshot'] = numscreenshot
    df.at[index, 'tags'] = tags
    df.at[index, 'editor_comment'] = editorcomment
    df.at[index, 'description_text'] = description
    df.at[index, 'inner_page_crawled'] = True
    
    log(f'[index]{index} [appname]{appname} [starpercent]{starpercent} [installcount]{installcount}')
    
    # write to csv every 30 entries
    global DONE
    DONE += 1
    if DONE % 30 == 0:
        log(f'>>> WRITING TO CSV AT DONE {DONE}')
        df.to_csv(FILENAME, index=False)
        log(f'>>> DONE WRITING TO CSV AT DONE {DONE}')
    
    # crawl at next index
#     global QUEUE
#     next_index = QUEUE.pop(0)
#     EXECUTOR.submit(crawl_inner_page, next_index, driver)
    DRIVERS.append(driver)


In [10]:
# capa = DesiredCapabilities.CHROME
# capa["pageLoadStrategy"] = "none"  # driver.get

# chrome_options = Options()
# chrome_options.add_argument("--disable-extensions")
# chrome_prefs = {
#       'profile.managed_default_content_settings.images': 2,
#     }
# chrome_options.add_experimental_option('prefs', chrome_prefs)

for i in QUEUE:
    EXECUTOR.submit(crawl_inner_page, i)

In [13]:
df.to_csv(FILENAME, index=False)